In [3]:
from skimage import img_as_float
import cv2
import numpy as np 
from skimage.feature import corner_harris, corner_subpix, corner_peaks
from skimage.morphology import skeletonize, thin
from math import pi,atan

# create a 3channel image 
# if color is white => symbol , black => background
# another matrix => visited pixels

#attempt algorithm to find corner 
#1.find suspected point as a begining of corner
#2.iterate from the right pixel
#3.if path from this pixel is longer than 4 and dominating direction is DOWN AND RIGHT and does not form a loop(closed path)
#  then this is a corner=> mark as visited
#4.repeat untill all white pixels are done

#return path length and top two dominating directions
def checkPathRightDown(y,x,visited,isCorner):
    #if valid path update visited and isCorner list
    w,h=visited.shape
    if(visited[y][x]):
        return
    count=0 
    nextX=x
    nextY=y
    path=[]
    path.append([nextX,nextY])
    while(nextX+1 != w and nextY+1 != h):
        #check right pixel
        if(skeleton[nextY][nextX+1]):
            nextX=nextX+1
            path.append([nextX,nextY])
        #check right down
        elif(skeleton[nextY+1][nextX+1]):
            nextX=nextX+1
            nextY=nextY+1
            path.append([nextX,nextY])
            count+=1
        #check down
        elif(skeleton[nextY+1][nextX]):
            nextY=nextY+1
            path.append([nextX,nextY])
        #if none exit loop
        else:
            break
    if(len(path)>=6):
        isCorner.append([x,y])
        for i in path:
            pX=i[0]
            pY=i[1]
            visited[pY][pX]=1

def findVinverted(skeleton):
    w,h=skeleton.shape
    visited = np.zeros((w,h))
    isCorner=[]
    index=0
    pixelIndices = np.where(skeleton==1)
    pixelCount =len(pixelIndices[0])
    for i in range(pixelCount):
        y = pixelIndices[0][i]
        x = pixelIndices[1][i]
        #check if in white pixel is on borders
        if(x+1 == w or y+1 == h or x== 0 or y==0):
            continue
        downR=skeleton[y+1][x+1]
        down=skeleton[y+1][x]
        right=skeleton[y][x+1]
        downL=skeleton[y+1][x-1]
        if (downR and downL):
            checkPathRightDown(y+1,x+1,visited,isCorner)
        elif (downR and down):
            checkPathRightDown(y+1,x+1,visited,isCorner)
        elif (right and downL):
            checkPathRightDown(y,x+1,visited,isCorner)
        elif (right and down):
            checkPathRightDown(y,x+1,visited,isCorner)
    return len(isCorner),visited





In [4]:
img = cv2.imread("11.png")
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 

#naive binarization

gray =np.where(gray<200,0,255)
gray=gray/255
gray=1-gray

skeleton = thin(gray)
cv2.imwrite("skeleton.png",skeleton*255)

countVinv,vInv=findVinverted(skeleton)

cv2.imwrite("potentialCorners.png",vInv*255)
print(countVinv)


NameError: name 'w' is not defined